# StealthGAN-IDS Training on Google Colab

This notebook provides a complete pipeline for training and evaluating StealthGAN-IDS on Google Colab.

**Features:**
- Automatic GPU detection and setup
- Dataset download and preprocessing
- Training with checkpointing
- Evaluation and visualization
- Easy result download

**Supported Datasets:**
- NSL-KDD (legacy)
- CIC-IDS2017
- CIC-IDS2018
- UNSW-NB15

## 1. Setup and Installation

In [1]:
# Check GPU availability
import torch
import os
from pathlib import Path

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected! Training will be very slow on CPU.")
    print("Go to Runtime > Change runtime type > Hardware accelerator > GPU")

CUDA available: True
GPU: NVIDIA GeForce RTX 5090
CUDA version: 12.8
GPU Memory: 33.66 GB


In [2]:
# Clone the repository
# Auto-detect environment (Colab vs QuickPod vs local)
import os
from pathlib import Path

# Detect base directory
if Path("/workspace").exists():
    # QuickPod environment
    base_dir = Path("/workspace")
elif Path("/content").exists():
    # Google Colab environment
    base_dir = Path("/content")
else:
    # Local or other environment
    base_dir = Path.cwd()

repo_dir = base_dir / "SGAN-IDS"
repo_url = "https://github.com/yourusername/SGAN-IDS.git"  # ⚠️ UPDATE THIS if cloning

print(f"Detected environment: {base_dir}")
print(f"Repository directory: {repo_dir}")

# Check if repo exists in current directory or base directory
if repo_dir.exists():
    print(f"✅ Repository found at {repo_dir}")
    os.chdir(repo_dir)
    if (repo_dir / ".git").exists():
        !git pull
elif Path("SGAN-IDS").exists():
    # Check if repo is in current working directory
    repo_dir = Path("SGAN-IDS").resolve()
    print(f"✅ Repository found at {repo_dir}")
    os.chdir(repo_dir)
elif Path.cwd().name == "SGAN-IDS":
    # Already in the repo directory
    repo_dir = Path.cwd()
    print(f"✅ Already in repository directory: {repo_dir}")
else:
    # Try to clone or use current directory
    if repo_url != "https://github.com/yourusername/SGAN-IDS.git":
        print(f"Cloning repository to {repo_dir}...")
        !git clone {repo_url} {repo_dir}
        os.chdir(repo_dir)
    else:
        # Use current directory as repo (for QuickPod where files are already there)
        repo_dir = Path.cwd()
        print(f"⚠️  Using current directory as repository: {repo_dir}")
        print("If this is wrong, update repo_url above or ensure SGAN-IDS folder exists")

print(f"Working directory: {os.getcwd()}")
print(f"Repository root: {repo_dir}")

Detected environment: /workspace
Repository directory: /workspace/SGAN-IDS
✅ Repository found at /workspace/SGAN-IDS
Already up to date.
Working directory: /workspace/SGAN-IDS
Repository root: /workspace/SGAN-IDS


In [3]:
# Install dependencies
# Check if torch is already installed with CUDA support (Colab has it pre-installed)
if not torch.cuda.is_available():
    print("Installing PyTorch with CUDA support...")
    !pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
else:
    print(f"PyTorch {torch.__version__} already installed with CUDA support ✓")

!pip install -q -r requirements.txt

# Optional: Install additional evaluation dependencies
!pip install -q xgboost lightgbm pyyaml

print("✅ Dependencies installed!")

PyTorch 2.9.1+cu128 already installed with CUDA support ✓
✅ Dependencies installed!


## 2. Dataset Setup

Choose your dataset and download/prepare it.

In [4]:
# Configuration
from pathlib import Path

# Auto-detect data root based on environment
if Path("/workspace").exists():
    DATA_ROOT = Path("/workspace/data")  # QuickPod
elif Path("/content").exists():
    DATA_ROOT = Path("/content/data")  # Colab
else:
    DATA_ROOT = Path("./data")  # Local

DATA_ROOT.mkdir(exist_ok=True, parents=True)

DATASET = "cic_ids2018"  # Options: nsl_kdd, cic_ids2017, cic_ids2018, unsw_nb15, unified

print(f"Selected dataset: {DATASET}")
print(f"Data root: {DATA_ROOT}")

Selected dataset: cic_ids2018
Data root: /workspace/data


In [5]:
# Download CIC-IDS2018 from Kaggle (requires Kaggle API)
# Option 1: Using Kaggle API (recommended)
if DATASET == "cic_ids2018":
    print("To download CIC-IDS2018:")
    print("1. Install Kaggle: !pip install kaggle")
    print("2. Upload kaggle.json (from Kaggle account settings)")
    print("3. Run: !mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json")
    print("4. Then run:")
    print(f"   !kaggle datasets download -d solarmainframe/ids-intrusion-csv -p {DATA_ROOT}")
    print(f"   !unzip -q {DATA_ROOT}/ids-intrusion-csv.zip -d {DATA_ROOT}/CIC-IDS2018")
    print("\n⚠️  Or manually upload dataset files via Colab file browser")

# Option 2: Manual upload via Colab file browser
# Files > Upload to session storage > Extract to DATA_ROOT

To download CIC-IDS2018:
1. Install Kaggle: !pip install kaggle
2. Upload kaggle.json (from Kaggle account settings)
3. Run: !mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
4. Then run:
   !kaggle datasets download -d solarmainframe/ids-intrusion-csv -p /workspace/data
   !unzip -q /workspace/data/ids-intrusion-csv.zip -d /workspace/data/CIC-IDS2018

⚠️  Or manually upload dataset files via Colab file browser


In [6]:
# Quick fix for QuickPod - set paths manually
import os
from pathlib import Path

# QuickPod paths
repo_dir = Path("/workspace/SGAN-IDS")
os.chdir(repo_dir)
print(f"Working directory: {os.getcwd()}")
print(f"Repository: {repo_dir}")

Working directory: /workspace/SGAN-IDS
Repository: /workspace/SGAN-IDS


In [7]:
!kaggle datasets download -d solarmainframe/ids-intrusion-csv -p {DATA_ROOT}

Dataset URL: https://www.kaggle.com/datasets/solarmainframe/ids-intrusion-csv
License(s): Attribution 4.0 International (CC BY 4.0)
 79%|██████████████████████████████        | 1.27G/1.60G [00:00<00:00, 4.54GB/s]
100%|██████████████████████████████████████| 1.60G/1.60G [00:00<00:00, 4.53GB/s]


In [8]:
!unzip -q {DATA_ROOT}/ids-intrusion-csv.zip -d {DATA_ROOT}/CIC-IDS2018

In [5]:
!python scripts/tune_optuna.py --data-root /workspace/data --dataset cic_ids2018 --n-trials 50 --max-samples 300000

StealthGAN-IDS Hyperparameter Tuning with Optuna
[tune] Using device: cuda

[1/3] Loading data...
[tune] Limiting dataset to 300000 samples
[CIC-IDS2018] Early stop after 1 files, ~450000 rows
[CIC-IDS2018] Sampling 300000 from 450000 rows
[CIC-IDS2018] Loaded shape: (300000, 80)
[DataForge] Loaded datasets: ['cic_ids2018']
[DataForge] Dropped 294 rows with NaN/inf values
[DataForge] Data shape after cleaning: (160627, 80)
[DataForge] Number of classes: 3
[DataForge] Classes: ['Benign', 'FTP-BruteForce', 'SSH-Bruteforce']
[DataForge] Split sizes - Train: 112438, Val: 24094, Test: 24095
[DataForge] Fitting transformers on training data...
[DataForge] Transforming validation data...
[DataForge] Transforming test data...
[DataForge] Converting sparse to dense...
[DataForge] Number of features after encoding: 18338
[DataForge] Reducing dimensions from 18338 to 256 using TruncatedSVD...
^C


In [9]:
# Preprocess the dataset (optional - training will preprocess if needed)
# Note: preprocess_data.py only supports: nsl_kdd, cic_ids2017, unified
# For cic_ids2018 and unsw_nb15, preprocessing happens automatically during training

import sys
import subprocess

# Only preprocess if dataset is supported by preprocess script
if DATASET in ["nsl_kdd", "cic_ids2017", "unified"]:
    print(f"Preprocessing {DATASET}...")
    
    cmd = [
        "python", "scripts/preprocess_data.py",
        "--data-root", str(DATA_ROOT),
        "--dataset", DATASET
    ]
    
    result = subprocess.run(cmd, cwd=repo_dir)
    
    if result.returncode == 0:
        print("✅ Preprocessing complete!")
    else:
        print(f"❌ Preprocessing failed with exit code {result.returncode}")
else:
    print(f"⚠️  Dataset {DATASET} will be preprocessed automatically during training")
    print("Skipping standalone preprocessing step...")

⚠️  Dataset cic_ids2018 will be preprocessed automatically during training
Skipping standalone preprocessing step...


## 3. Training

Configure training parameters and start training.

In [14]:
# Training configuration
EPOCHS = 100
BATCH_SIZE = 256  # Reduced for Colab memory (default is 256, use 64-128 for Colab)
MAX_SAMPLES = 500000  # Limit dataset size for Colab RAM (None = use all, 500k works well)
CHECKPOINT_INTERVAL = 5  # Save checkpoint every N epochs
SEED = 42
USE_AMP = True  # Mixed precision (faster on modern GPUs)
NUM_WORKERS = 2  # Reduce if you get errors

# Clear GPU memory before training
import gc
import torch
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print(f"GPU memory cleared. Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

print(f"\nTraining configuration:")
print(f"  Dataset: {DATASET}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Max samples: {MAX_SAMPLES if MAX_SAMPLES else 'all'}")
print(f"  Epochs: {EPOCHS}")
print(f"  Mixed precision: {USE_AMP}")
print(f"  Seed: {SEED}")

GPU memory cleared. Available: 33.66 GB

Training configuration:
  Dataset: cic_ids2018
  Batch size: 256
  Max samples: 250000
  Epochs: 100
  Mixed precision: True
  Seed: 42


In [15]:
# Start training
import subprocess

cmd = [
    "python", "scripts/train_gan.py",
    "--data-root", str(DATA_ROOT),
    "--dataset", DATASET,
    "--epochs", str(EPOCHS),
    "--batch-size", str(BATCH_SIZE),
    "--checkpoint-interval", str(CHECKPOINT_INTERVAL),
    "--seed", str(SEED),
    "--num-workers", str(NUM_WORKERS),
]

if MAX_SAMPLES:
    cmd.extend(["--max-samples", str(MAX_SAMPLES)])

if USE_AMP:
    cmd.append("--amp")

print("Starting training...")
print(f"Command: {' '.join(cmd)}")
print("\n⚠️  This may take several hours. Checkpoints will be saved periodically.")
print("💡 If you get OOM errors, reduce MAX_SAMPLES (try 200000) or BATCH_SIZE (try 32)")

# Run training
result = subprocess.run(cmd, cwd=repo_dir)

if result.returncode == 0:
    print("\n✅ Training completed successfully!")
elif result.returncode == -9:
    print("\n❌ Training killed (exit code -9) - Out of Memory!")
    print("   Try reducing MAX_SAMPLES to 200000 or BATCH_SIZE to 32")
else:
    print(f"\n❌ Training failed with exit code {result.returncode}")

Starting training...
Command: python scripts/train_gan.py --data-root /workspace/data --dataset cic_ids2018 --epochs 100 --batch-size 256 --checkpoint-interval 5 --seed 42 --num-workers 2 --max-samples 250000 --amp

⚠️  This may take several hours. Checkpoints will be saved periodically.
💡 If you get OOM errors, reduce MAX_SAMPLES (try 200000) or BATCH_SIZE (try 32)


KeyboardInterrupt: 

In [9]:
!python scripts/train_gan.py --data-root /workspace/data --dataset cic_ids2018 --epochs 100 --checkpoint-interval 5 --seed 42 --num-workers 2 --max-samples 500000 --amp

[train] Using device: cuda
[train] Limiting dataset to 500000 samples
[CIC-IDS2018] Early stop after 1 files, ~750000 rows
[CIC-IDS2018] Sampling 500000 from 750000 rows
[CIC-IDS2018] Loaded shape: (500000, 80)
[DataForge] Loaded datasets: ['cic_ids2018']
[DataForge] Dropped 1307 rows with NaN/inf values
[DataForge] Data shape after cleaning: (359590, 80)
[DataForge] Number of classes: 3
[DataForge] Classes: ['Benign', 'FTP-BruteForce', 'SSH-Bruteforce']
[DataForge] Split sizes - Train: 251712, Val: 53939, Test: 53939
[DataForge] Fitting transformers on training data...
[DataForge] Transforming validation data...
[DataForge] Transforming test data...
[DataForge] Converting sparse to dense...
[DataForge] Number of features after encoding: 29093
[DataForge] Reducing dimensions from 29093 to 256 using TruncatedSVD...
Killed


In [27]:
# Check training outputs
output_files = [
    "training_stats.csv",
    "generator_best.pth",
    "generator_ema_best.pth",
    "generator.pth",
    "discriminator.pth",
]

print("Training outputs:")
for fname in output_files:
    path = repo_dir / fname
    if path.exists():
        size_mb = path.stat().st_size / 1e6
        print(f"  ✅ {fname} ({size_mb:.2f} MB)")
    else:
        print(f"  ❌ {fname} (not found)")

# List checkpoints
checkpoints = list(repo_dir.glob("checkpoint_epoch_*.pth"))
if checkpoints:
    print(f"\nCheckpoints found: {len(checkpoints)}")
    for cp in sorted(checkpoints)[-5:]:  # Show last 5
        size_mb = cp.stat().st_size / 1e6
        print(f"  {cp.name} ({size_mb:.2f} MB)")

Training outputs:
  ✅ training_stats.csv (0.00 MB)
  ✅ generator_best.pth (4.61 MB)
  ✅ generator_ema_best.pth (4.61 MB)
  ✅ generator.pth (4.61 MB)
  ✅ discriminator.pth (0.58 MB)

Checkpoints found: 7
  checkpoint_epoch_20.pth (20.19 MB)
  checkpoint_epoch_25.pth (20.19 MB)
  checkpoint_epoch_30.pth (20.19 MB)
  checkpoint_epoch_35.pth (20.19 MB)
  checkpoint_epoch_5.pth (20.19 MB)


## 4. Evaluation

Evaluate the trained generator with comprehensive metrics.

In [28]:
# Evaluation configuration
GENERATOR_PATH = "generator_ema_best.pth"  # Use EMA version (better quality)
N_PER_CLASS = 2000  # Synthetic samples per class
TARGET_MINORITY = True  # Focus on minority classes
CV_FOLDS = 5  # Cross-validation folds
OUTPUT_DIR = "eval_outputs"

print(f"Evaluation configuration:")
print(f"  Generator: {GENERATOR_PATH}")
print(f"  Samples per class: {N_PER_CLASS}")
print(f"  Target minority: {TARGET_MINORITY}")
print(f"  CV folds: {CV_FOLDS}")

Evaluation configuration:
  Generator: generator_ema_best.pth
  Samples per class: 2000
  Target minority: True
  CV folds: 5


In [ ]:
# Run evaluation
import subprocess
import gc
import torch

# Clear memory before evaluation
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

cmd = [
    "python", "scripts/eval_gan.py",
    "--data-root", str(DATA_ROOT),
    "--dataset", DATASET,
    "--generator-path", GENERATOR_PATH,
    "--n-per-class", str(N_PER_CLASS),
    "--cv-folds", str(CV_FOLDS),
    "--output-dir", OUTPUT_DIR,
]

# Pass max_samples to prevent OOM during data loading
if MAX_SAMPLES:
    cmd.extend(["--max-samples", str(MAX_SAMPLES)])

if TARGET_MINORITY:
    cmd.append("--target-minority")

print("Starting evaluation...")
print(f"Command: {' '.join(cmd)}")

result = subprocess.run(cmd, cwd=repo_dir)

if result.returncode == 0:
    print("\n✅ Evaluation completed successfully!")
elif result.returncode == -9:
    print("\n❌ Evaluation killed (exit code -9) - Out of Memory!")
    print("   Try reducing MAX_SAMPLES to 200000 or N_PER_CLASS to 1000")
else:
    print(f"\n❌ Evaluation failed with exit code {result.returncode}")

Starting evaluation...
Command: python scripts/eval_gan.py --data-root /workspace/data --dataset cic_ids2018 --generator-path generator_ema_best.pth --n-per-class 2000 --cv-folds 5 --output-dir eval_outputs --max-samples 250000 --target-minority


In [31]:
# View evaluation results
import json

results_file = repo_dir / OUTPUT_DIR / "evaluation_results.json"
if results_file.exists():
    with open(results_file) as f:
        results = json.load(f)
    
    print("Evaluation Results Summary:")
    print("=" * 50)
    
    if "classifier_results" in results:
        print("\nClassifier Performance:")
        for classifier, metrics in results["classifier_results"].items():
            if "baseline" in metrics and "augmented" in metrics:
                baseline = metrics["baseline"]["mean"]
                augmented = metrics["augmented"]["mean"]
                improvement = augmented - baseline
                print(f"  {classifier}:")
                print(f"    Baseline F1: {baseline:.4f}")
                print(f"    Augmented F1: {augmented:.4f}")
                print(f"    Improvement: {improvement:+.4f}")
    
    if "distribution_metrics" in results:
        print("\nDistribution Quality:")
        for metric, value in results["distribution_metrics"].items():
            print(f"  {metric}: {value:.4f}")
else:
    print("⚠️  Results file not found. Run evaluation first.")

⚠️  Results file not found. Run evaluation first.


## 5. Visualizations

View generated plots and visualizations.

In [ ]:
# Display evaluation plots
from IPython.display import Image, display

plots_dir = repo_dir / OUTPUT_DIR / "plots"
if plots_dir.exists():
    plot_files = list(plots_dir.glob("*.png"))
    if plot_files:
        print(f"Found {len(plot_files)} plot(s):")
        for plot_file in plot_files:
            print(f"\n{plot_file.name}:")
            display(Image(str(plot_file)))
    else:
        print("No plots found in plots directory.")
else:
    print("Plots directory not found. Run evaluation first.")

## 6. Download Results

Download your trained models and results.

In [ ]:
# Create download package
import shutil
from pathlib import Path

download_dir = Path("/content/downloads")
download_dir.mkdir(exist_ok=True)

# Copy important files
files_to_download = [
    "generator_ema_best.pth",
    "generator_best.pth",
    "discriminator.pth",
    "training_stats.csv",
]

# Copy checkpoints
checkpoints = list(repo_dir.glob("checkpoint_epoch_*.pth"))
if checkpoints:
    checkpoint_dir = download_dir / "checkpoints"
    checkpoint_dir.mkdir(exist_ok=True)
    for cp in checkpoints:
        shutil.copy2(cp, checkpoint_dir / cp.name)
    print(f"Copied {len(checkpoints)} checkpoints")

# Copy evaluation outputs
eval_dir = repo_dir / OUTPUT_DIR
if eval_dir.exists():
    shutil.copytree(eval_dir, download_dir / OUTPUT_DIR, dirs_exist_ok=True)
    print("Copied evaluation outputs")

# Copy files
for fname in files_to_download:
    src = repo_dir / fname
    if src.exists():
        shutil.copy2(src, download_dir / fname)

print(f"\n✅ Files prepared for download in {download_dir}")
print("\nTo download:")
print("1. Use Colab file browser (left sidebar)")
print("2. Navigate to /content/downloads")
print("3. Right-click files and select 'Download'")

In [ ]:
# Alternative: Create a zip file for easy download
import shutil
from pathlib import Path
from google.colab import files

zip_path = "/content/stealthgan_results.zip"
download_dir = Path("/content/downloads")

if download_dir.exists():
    shutil.make_archive(
        zip_path.replace(".zip", ""),
        "zip",
        download_dir
    )
    
    size_mb = Path(zip_path).stat().st_size / 1e6
    print(f"✅ Created zip file: {zip_path} ({size_mb:.2f} MB)")
    print("\nDownloading zip file...")
    files.download(zip_path)
else:
    print("⚠️  Download directory not found. Run the previous cell first.")

## 7. Resume Training (Optional)

Resume training from a checkpoint if your session disconnects.

In [ ]:
# Resume training from checkpoint
CHECKPOINT_PATH = "checkpoint_epoch_50.pth"  # ⚠️ Update with your checkpoint name
RESUME_EPOCHS = 100  # Total epochs (will continue from checkpoint)

cmd = [
    "python", "scripts/train_gan.py",
    "--data-root", str(DATA_ROOT),
    "--dataset", DATASET,
    "--epochs", str(RESUME_EPOCHS),
    "--batch-size", str(BATCH_SIZE),
    "--resume", CHECKPOINT_PATH,
    "--seed", str(SEED),
    "--num-workers", str(NUM_WORKERS),
]

if MAX_SAMPLES:
    cmd.extend(["--max-samples", str(MAX_SAMPLES)])

if USE_AMP:
    cmd.append("--amp")

print("To resume training, update CHECKPOINT_PATH above and uncomment the last line:")
print(f"{' '.join(cmd)}")

# Uncomment to run:
# result = subprocess.run(cmd, cwd=repo_dir)

## Troubleshooting

**Common Issues:**

1. **Out of Memory (exit code -9)**:
   - Reduce `MAX_SAMPLES` (try 200000 or 100000)
   - Reduce `BATCH_SIZE` (try 32 or 16)
   - Use a smaller dataset (`nsl_kdd` instead of `cic_ids2018`)
2. **Session Timeout**: Colab free tier has 12hr limit. Use checkpoints to resume.
3. **Dataset Not Found**: Ensure dataset is downloaded and extracted correctly.
4. **Slow Training**: Enable GPU (Runtime > Change runtime type > GPU)

**Memory Guide for Colab Free Tier (12GB RAM):**
| Dataset | Recommended MAX_SAMPLES |
|---------|------------------------|
| NSL-KDD | None (all ~125K) |
| CIC-IDS2017 | 500000 |
| CIC-IDS2018 | 500000 |
| UNSW-NB15 | None (all ~175K) |

**Tips:**
- Save checkpoints frequently
- Use Colab Pro for longer sessions (24hr limit) and more RAM
- Download results before session expires
- Monitor GPU usage: `!nvidia-smi`

In [ ]:
# Monitor GPU usage
!nvidia-smi